In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [10]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [11]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [12]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
# We define our neural network by subclassing nn.Module, 
# and initialize the neural network layers in __init__. Every nn.Module 
# subclass implements the operations on input data in the forward method.

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#We create an instance of NeuralNetwork, and move it to the device, and print its structure.
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


**Model Layers**


Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [13]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


**nn.Flatten**


We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [14]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


**nn.Linear**


The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [15]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


**nn.ReLU**


Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [16]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1438, -0.6433, -0.0899, -0.1633,  0.2091,  0.3837,  0.1641, -0.1010,
         -0.5305,  0.0507, -0.3045, -1.0164, -0.2054, -0.1427, -0.3975,  0.3420,
         -0.6190,  0.5097,  0.0481, -0.3499],
        [-0.2474, -0.5045, -0.0569, -0.0283, -0.0499,  0.0024,  0.1480, -0.3763,
         -0.3210, -0.0296, -0.6428, -1.0387, -0.2547, -0.3501, -0.5454,  0.3916,
         -0.6317,  0.6768,  0.1302, -0.5435],
        [-0.0924, -0.7572, -0.2102,  0.0287,  0.1669, -0.1273,  0.0281, -0.3341,
         -0.4838, -0.1241, -0.2140, -1.1430, -0.3336,  0.0948, -0.4752,  0.4528,
         -0.5914,  0.5572,  0.4154, -0.3894]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1438, 0.0000, 0.0000, 0.0000, 0.2091, 0.3837, 0.1641, 0.0000, 0.0000,
         0.0507, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3420, 0.0000, 0.5097,
         0.0481, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0024, 0.1480, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

**nn.Sequential**


nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [17]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

**nn.Softmax**


The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [18]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

**Model Parameters**


Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [19]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0312, -0.0201, -0.0088,  ..., -0.0080, -0.0104,  0.0056],
        [-0.0066, -0.0132, -0.0015,  ..., -0.0265, -0.0179,  0.0317]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0239, -0.0065], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0441,  0.0128,  0.0178,  ..., -0.0013, -0.0425, -0.0016],
        [ 0.0174,  0.0216,  0.0418,  ...,  0.0359,  0.0026, -0.0047]],
       device='cuda:0', grad_fn=<

**Optimizing the Model Parameters**


To train a model, we need a loss function and an optimizer.

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model’s performance against the test dataset to ensure it is learning.

In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [23]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314404  [    0/60000]
loss: 2.299270  [ 6400/60000]
loss: 2.278633  [12800/60000]
loss: 2.260244  [19200/60000]
loss: 2.254174  [25600/60000]
loss: 2.224105  [32000/60000]
loss: 2.224135  [38400/60000]
loss: 2.194371  [44800/60000]
loss: 2.182723  [51200/60000]
loss: 2.145499  [57600/60000]
Test Error: 
 Accuracy: 50.3%, Avg loss: 2.147711 

Epoch 2
-------------------------------
loss: 2.161387  [    0/60000]
loss: 2.150744  [ 6400/60000]
loss: 2.094269  [12800/60000]
loss: 2.095822  [19200/60000]
loss: 2.061633  [25600/60000]
loss: 1.992745  [32000/60000]
loss: 2.013618  [38400/60000]
loss: 1.933617  [44800/60000]
loss: 1.932984  [51200/60000]
loss: 1.851065  [57600/60000]
Test Error: 
 Accuracy: 60.0%, Avg loss: 1.864996 

Epoch 3
-------------------------------
loss: 1.904181  [    0/60000]
loss: 1.872984  [ 6400/60000]
loss: 1.760800  [12800/60000]
loss: 1.780962  [19200/60000]
loss: 1.702180  [25600/60000]
loss: 1.641765  [32000/600

**Saving Models**


A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [24]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


**Loading Models**


The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [25]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.

In [26]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
